In [7]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QV_4_5it.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QV_4_5it.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 4

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        circuit = QuantumVolume(num_qubits, depth=num_qubits)
        # circuit = QFT(num_qubits, do_swaps=False)
        # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 4
Mean Costs:
  Recursive:84.800
Mean Times (s):
  Recursive:0.024
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 4
Mean Costs:
  Recursive:181.800
Mean Times (s):
  Recursive:0.124
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 4
Mean Costs:
  Recursive:315.800
Mean Times (s):
  Recursive:0.322
Finished 10 iterations for:
  # Qubits: 40, # Partitions: 4
Mean Costs:
  Recursive:490.400
Mean Times (s):
  Recursive:0.828
Finished 10 iterations for:
  # Qubits: 48, # Partitions: 4
Mean Costs:
  Recursive:691.000
Mean Times (s):
  Recursive:1.854
Finished 10 iterations for:
  # Qubits: 56, # Partitions: 4
Mean Costs:
  Recursive:933.800
Mean Times (s):
  Recursive:3.742
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 4
Mean Costs:
  Recursive:1214.800
Mean Times (s):
  Recursive:7.030
Finished 10 iterations for:
  # Qubits: 72, # Partitions: 4
Mean Costs:
  Recursive:1520.800
Mean Times (s):
  Recursi

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QFT_5it_4part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QFT_5it_4part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        # circuit = QFT(num_qubits, do_swaps=False)
        circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:22.800
Mean Times (s):
  Recursive:0.007
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:40.000
Mean Times (s):
  Recursive:0.024
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:48.400
Mean Times (s):
  Recursive:0.088
Finished 10 iterations for:
  # Qubits: 40, # Partitions: 2
Mean Costs:
  Recursive:60.400
Mean Times (s):
  Recursive:0.137
Finished 10 iterations for:
  # Qubits: 48, # Partitions: 2
Mean Costs:
  Recursive:77.200
Mean Times (s):
  Recursive:0.250
Finished 10 iterations for:
  # Qubits: 56, # Partitions: 2
Mean Costs:
  Recursive:94.400
Mean Times (s):
  Recursive:0.445
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 2
Mean Costs:
  Recursive:87.600
Mean Times (s):
  Recursive:0.620
Finished 10 iterations for:
  # Qubits: 72, # Partitions: 2
Mean Costs:
  Recursive:106.400
Mean Times (s):
  Recursive:1.080

In [6]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QAOA_5it_3part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QAOA_5it_3part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 3

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        # circuit = QFT(num_qubits, do_swaps=False)
        circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

IndexError: list index out of range

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QAOA_5it_4part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QAOA_5it_4part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 4

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        # circuit = QFT(num_qubits, do_swaps=False)
        circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 4
Mean Costs:
  Recursive:62.000
Mean Times (s):
  Recursive:0.011
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 4
Mean Costs:
  Recursive:101.400
Mean Times (s):
  Recursive:0.035
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 4
Mean Costs:
  Recursive:147.200
Mean Times (s):
  Recursive:0.103
Finished 10 iterations for:
  # Qubits: 40, # Partitions: 4
Mean Costs:
  Recursive:191.000
Mean Times (s):
  Recursive:0.228
Finished 10 iterations for:
  # Qubits: 48, # Partitions: 4
Mean Costs:
  Recursive:221.400
Mean Times (s):
  Recursive:0.414
Finished 10 iterations for:
  # Qubits: 56, # Partitions: 4
Mean Costs:
  Recursive:270.600
Mean Times (s):
  Recursive:0.732
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 4
Mean Costs:
  Recursive:291.000
Mean Times (s):
  Recursive:1.143
Finished 10 iterations for:
  # Qubits: 72, # Partitions: 4
Mean Costs:
  Recursive:354.400
Mean Times (s):
  Recursive

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QFT_5it_2part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QFT_5it_2part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        circuit = QFT(num_qubits, do_swaps=False)
        # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 2
Mean Costs:
  Recursive:32.000
Mean Times (s):
  Recursive:0.009
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 2
Mean Costs:
  Recursive:48.000
Mean Times (s):
  Recursive:0.031
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 2
Mean Costs:
  Recursive:64.000
Mean Times (s):
  Recursive:0.086
Finished 10 iterations for:
  # Qubits: 40, # Partitions: 2
Mean Costs:
  Recursive:80.000
Mean Times (s):
  Recursive:0.199
Finished 10 iterations for:
  # Qubits: 48, # Partitions: 2
Mean Costs:
  Recursive:96.000
Mean Times (s):
  Recursive:0.396
Finished 10 iterations for:
  # Qubits: 56, # Partitions: 2
Mean Costs:
  Recursive:112.000
Mean Times (s):
  Recursive:0.745
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 2
Mean Costs:
  Recursive:128.000
Mean Times (s):
  Recursive:1.247
Finished 10 iterations for:
  # Qubits: 72, # Partitions: 2
Mean Costs:
  Recursive:144.000
Mean Times (s):
  Recursive:1.9

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QFT_5it_3part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QFT_5it_3part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 3

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        circuit = QFT(num_qubits, do_swaps=False)
        # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

IndexError: list index out of range

In [8]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QFT_5it_4part.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QFT_5it_4part.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 4

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        circuit = QFT(num_qubits, do_swaps=False)
        # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  FGP:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  FGP:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 16, # Partitions: 4
Mean Costs:
  FGP:72.000
Mean Times (s):
  FGP:0.016
Finished 10 iterations for:
  # Qubits: 24, # Partitions: 4
Mean Costs:
  FGP:112.000
Mean Times (s):
  FGP:0.061
Finished 10 iterations for:
  # Qubits: 32, # Partitions: 4
Mean Costs:
  FGP:148.000
Mean Times (s):
  FGP:0.185
Finished 10 iterations for:
  # Qubits: 40, # Partitions: 4
Mean Costs:
  FGP:188.000
Mean Times (s):
  FGP:0.464
Finished 10 iterations for:
  # Qubits: 48, # Partitions: 4
Mean Costs:
  FGP:224.000
Mean Times (s):
  FGP:0.848
Finished 10 iterations for:
  # Qubits: 56, # Partitions: 4
Mean Costs:
  FGP:264.000
Mean Times (s):
  FGP:1.576
Finished 10 iterations for:
  # Qubits: 64, # Partitions: 4
Mean Costs:
  FGP:304.000
Mean Times (s):
  FGP:2.612
Finished 10 iterations for:
  # Qubits: 72, # Partitions: 4
Mean Costs:
  FGP:340.000
Mean Times (s):
  FGP:4.255
Finished 10 iterations for:
  # Qubits: 80, # Partitions: 4
Mean Costs:
  FGP:407.000
Mea

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time
from cp_fraction import cp_fraction

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_QFT_5it.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_QFT_5it.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(16, 97, 8)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2 + i

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5)
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        # circuit = QFT(num_qubits, do_swaps=False)
        # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

In [ ]:
import os
import json
import time
import numpy as np
from qiskit import transpile
from qiskit.circuit.library import QFT, QuantumVolume
from QAOA import QAOA_random
from fgp_roee import *
from qiskit.circuit.library import QuantumVolume, QFT
from qiskit import transpile
import time
from cp_fraction import cp_fraction

###############################################################################
# Set up JSON file for storing *all* iteration results (detailed data)
###############################################################################
detailed_filename = "benchmark_results_FGP_CP_large_5it.json"

if os.path.exists(detailed_filename):
    with open(detailed_filename, "r") as f:
        detailed_results = json.load(f)
else:
    detailed_results = []

###############################################################################
# Set up JSON file for *aggregated* results (mean cost/time)
###############################################################################
means_filename = "benchmark_means_FGP_CP_large_5it.json"

if os.path.exists(means_filename):
    with open(means_filename, "r") as f:
        mean_results = json.load(f)
else:
    mean_results = []

sizes = range(256, 257, 16)

for i, num_qubits in enumerate(sizes):
    # For each increase of 8 qubits, increase the number of partitions by 1
    num_partitions = 2

    # Create an All-to-All network
    qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
    
    # Sweep the fraction parameter from 0.1 to 0.9
        # Collect data for computing means across 10 iterations
    iteration_data = []
    for iteration in range(5):
        
        # -------------------------
        # 1. Define/redefine circuit
        # -------------------------

        # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
        # Define the number of qubits and the circuit
        circuit = cp_fraction(num_qubits, num_qubits, fraction=0.5)
        # circuit = QuantumVolume(num_qubits, depth=num_qubits)
        # circuit = QFT(num_qubits, do_swaps=False)
        # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
        # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
        basis_gates = ['cp', 'u']
        # basis_gates = ['cp','u']
        transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
        # Define the number of partitions
        # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
        # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
        # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

        qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


        initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
        start = time.time()
        partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
        stop = time.time()

        total_time_fgp = stop - start
        min_cost_fgp = cost

        
        # -------------------------
        # 6. Store iteration-level results
        # -------------------------
        result_entry = {
            "num_qubits": num_qubits,
            "num_partitions": num_partitions,
            "iteration": iteration,
            "fgp_cost":  min_cost_fgp,
            "time_fgp": total_time_fgp,
        }
        
        detailed_results.append(result_entry)
        iteration_data.append(result_entry)
        
        # Update detailed JSON right away
        with open(detailed_filename, "w") as f:
            json.dump(detailed_results, f, indent=2)
    
    # ---------------------------------------------------------------------
    # After 10 iterations, compute the means and log them
    # ---------------------------------------------------------------------

    r_cost_list = [x["fgp_cost"] for x in iteration_data]
    

    r_time_list = [x["time_fgp"] for x in iteration_data]
    

    mean_r_cost = float(np.mean(r_cost_list))
    

    mean_r_time = float(np.mean(r_time_list))
    
    # Print to console for quick logging
    print("=============================================")
    print(f"Finished 10 iterations for:")
    print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions}")
    print("Mean Costs:")

    print(f"  Recursive:{mean_r_cost:.3f}")
    print("Mean Times (s):")

    print(f"  Recursive:{mean_r_time:.3f}")
    print("=============================================")
    
    # Store the aggregated means in a separate JSON
    mean_entry = {
        "num_qubits": num_qubits,
        "num_partitions": num_partitions,

        "mean_r_cost": mean_r_cost,

        "mean_r_time": mean_r_time,
    }
    
    mean_results.append(mean_entry)
    
    # Update the means JSON file
    with open(means_filename, "w") as f:
        json.dump(mean_results, f, indent=2)

print("Benchmarking completed. Detailed results saved to", detailed_filename)
print("Aggregated means saved to", means_filename)

KeyboardInterrupt: 

In [ ]:
# import os
# import json
# import time
# import numpy as np
# from qiskit import transpile
# from qiskit.circuit.library import QFT, QuantumVolume
# from QAOA import QAOA_random
# from cp_fraction import cp_fraction
# from fgp_roee import *
# from qiskit.circuit.library import QuantumVolume, QFT
# from qiskit import transpile
# import time


# ###############################################################################
# # Set up JSON file for storing *all* iteration results (detailed data)
# ###############################################################################
# detailed_filename = "benchmark_results_FGP_CP_5it.json"

# if os.path.exists(detailed_filename):
#     with open(detailed_filename, "r") as f:
#         detailed_results = json.load(f)
# else:
#     detailed_results = []

# ###############################################################################
# # Set up JSON file for *aggregated* results (mean cost/time)
# ###############################################################################
# means_filename = "benchmark_means_FGP_CP_5it.json"

# if os.path.exists(means_filename):
#     with open(means_filename, "r") as f:
#         mean_results = json.load(f)
# else:
#     mean_results = []

# fraction_list = [0.3, 0.5, 0.7, 0.9]
# sizes = range(96, 97, 8)

# for fraction in fraction_list:
#     for i, num_qubits in enumerate(sizes):
#         # For each increase of 8 qubits, increase the number of partitions by 1
#         num_partitions = 12 + i

#         # Create an All-to-All network
#         qpu_info = [int(num_qubits / num_partitions) + 1 for _ in range(num_partitions)]
        
#         # Sweep the fraction parameter from 0.1 to 0.9
#             # Collect data for computing means across 10 iterations
#         iteration_data = []
#         for iteration in range(5):
            
#             # -------------------------
#             # 1. Define/redefine circuit
#             # -------------------------

#             # base_graph = build_circuit(num_qubits,fraction=fraction,group_gates=True)
#             # Define the number of qubits and the circuit
#             circuit = QuantumVolume(num_qubits, depth=num_qubits)
#             circuit = cp_fraction(num_qubits, num_qubits, fraction=fraction)
#             # circuit = QFT(num_qubits, do_swaps=False)
#             # circuit = QAOA_random(num_qubits, prob=0.5, reps=1)
#             # Transpile the circuit into some basis gates. The gate set here was used to match those used in teh GCP paper, but it shouldn't matter which gates are used.
#             basis_gates = ['cp', 'u']
#             # basis_gates = ['cp','u']
#             transpiled_circuit = transpile(circuit, basis_gates=basis_gates)
#             # Define the number of partitions
#             # Define the QPU sizes in terms of data qubit capacity, here they are defined to be equal and match the number of qubits in the circuit.
#             # Note that if the number of qubits in the circuit is odd, fully local partitions can be impossible. 
#             # E.g. if you have a 9 qubit circuit and 3x3 qubit QPUs, then you can't accomodate 4 pairs of qubits interacting at the same time, so you need to increase the size of the QPUs.

#             qpu_sizes = [int(num_qubits/num_partitions) for _ in range(num_partitions)]


#             initial_partition = set_initial_partition(qpu_info=qpu_sizes,num_partitions=num_partitions)
#             start = time.time()
#             partition, cost, mapping = main_algorithm(circuit=transpiled_circuit, qpu_info=qpu_sizes,initial_partition=initial_partition,remove_singles=False,choose_initial=True)
#             stop = time.time()

#             total_time_fgp = stop - start
#             min_cost_fgp = cost

            
#             # -------------------------
#             # 6. Store iteration-level results
#             # -------------------------
#             result_entry = {
#                 "num_qubits": num_qubits,
#                 "num_partitions": num_partitions,
#                 "fraction" : fraction,
#                 "iteration": iteration,
#                 "fgp_cost":  min_cost_fgp,
#                 "time_fgp": total_time_fgp,
#             }
            
#             detailed_results.append(result_entry)
#             iteration_data.append(result_entry)
            
#             # Update detailed JSON right away
#             with open(detailed_filename, "w") as f:
#                 json.dump(detailed_results, f, indent=2)
        
#         # ---------------------------------------------------------------------
#         # After 10 iterations, compute the means and log them
#         # ---------------------------------------------------------------------

#         r_cost_list = [x["fgp_cost"] for x in iteration_data]
        

#         r_time_list = [x["time_fgp"] for x in iteration_data]
        

#         mean_r_cost = float(np.mean(r_cost_list))
        

#         mean_r_time = float(np.mean(r_time_list))
        
#         # Print to console for quick logging
#         print("=============================================")
#         print(f"Finished 10 iterations for:")
#         print(f"  # Qubits: {num_qubits}, # Partitions: {num_partitions} , Fraction: {fraction}")
#         print("Mean Costs:")

#         print(f"  Recursive:{mean_r_cost:.3f}")
#         print("Mean Times (s):")

#         print(f"  Recursive:{mean_r_time:.3f}")
#         print("=============================================")
        
#         # Store the aggregated means in a separate JSON
#         mean_entry = {
#             "num_qubits": num_qubits,
#             "num_partitions": num_partitions,
#             "fraction" : fraction,
#             "mean_r_cost": mean_r_cost,

#             "mean_r_time": mean_r_time,
#         }
        
#         mean_results.append(mean_entry)
        
#         # Update the means JSON file
#         with open(means_filename, "w") as f:
#             json.dump(mean_results, f, indent=2)

# print("Benchmarking completed. Detailed results saved to", detailed_filename)
# print("Aggregated means saved to", means_filename)

Finished 10 iterations for:
  # Qubits: 96, # Partitions: 12 , Fraction: 0.3
Mean Costs:
  Recursive:1511.200
Mean Times (s):
  Recursive:6.228
Finished 10 iterations for:
  # Qubits: 96, # Partitions: 12 , Fraction: 0.5
Mean Costs:
  Recursive:2436.400
Mean Times (s):
  Recursive:14.381
Finished 10 iterations for:
  # Qubits: 96, # Partitions: 12 , Fraction: 0.7
Mean Costs:
  Recursive:3301.600
Mean Times (s):
  Recursive:25.213
Finished 10 iterations for:
  # Qubits: 96, # Partitions: 12 , Fraction: 0.9
Mean Costs:
  Recursive:3942.000
Mean Times (s):
  Recursive:37.991
Benchmarking completed. Detailed results saved to benchmark_results_FGP_CP_5it.json
Aggregated means saved to benchmark_means_FGP_CP_5it.json
